# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import warnings

# Import API key
from api_keys import geoapify_key

In [46]:
# Warning filters
warnings.filterwarnings("ignore")

In [47]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tazovsky,67.4667,78.7000,3.47,78,96,10.00,RU,1727752105
1,1,invercargill,-46.4000,168.3500,13.34,79,100,1.34,NZ,1727752106
2,2,blackmans bay,-43.0167,147.3167,16.07,33,6,3.13,AU,1727752107
3,3,kalifornsky,60.4183,-151.2900,11.33,84,100,1.67,US,1727751816
4,4,enewetak,11.3474,162.3373,29.04,75,18,4.73,MH,1727752109


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [49]:
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1.0,
    size="Humidity",
    color = "City"
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [51]:
#  Narrow down cities that fit the criteria
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] < 27) &                 # Max Temp lower than 27
    (city_data_df['Max Temp'] > 21) &                 # Max Temp higher than 21
    (city_data_df['Wind Speed'] < 4.5)                # Wind Speed less than 4.5 m/s             
]

 # Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
sample_size = min(20, len(filtered_cities_df))  # Ensure sample size does not exceed available data
sample_data = filtered_cities_df.sample(sample_size)  
display(sample_data)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
476,476,tionk essil,12.7856,-16.5217,25.59,87,99,0.94,SN,1727752643
296,296,consolacion del sur,22.5047,-83.5136,25.28,93,58,2.06,CU,1727752441
349,349,sao tome,0.3365,6.7273,25.45,83,86,3.34,ST,1727752502
505,505,holualoa,19.6228,-155.9522,25.31,83,18,1.80,US,1727752675
333,333,les cayes,18.2000,-73.7500,26.42,82,0,1.48,HT,1727752483
259,259,mawlaik,23.6333,94.4167,26.16,86,100,0.33,MM,1727752396
453,453,ghat,24.9647,10.1728,25.97,32,0,2.04,LY,1727752619
245,245,vila velha,-20.3297,-40.2925,21.64,97,20,1.34,BR,1727752165
362,362,hadibu,12.6500,54.0333,25.23,83,12,2.43,YE,1727752516
178,178,tura,25.5198,90.2201,26.73,94,100,1.66,IN,1727752303


### Step 3: Create a new DataFrame called `hotel_df`.

In [53]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Lat', 'Lng','Humidity','Country']].copy()  # Adjust based on available columns

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
sample_hotel_data = hotel_df.sample(20) 
display(sample_hotel_data)

,City,Lat,Lng,Humidity,Country,Hotel Name
568,alvaraes,-3.2208,-64.8042,87,BR,
102,puerto armuelles,8.2833,-82.8667,88,PA,
223,ta`u,-14.2336,-169.5144,90,AS,
505,holualoa,19.6228,-155.9522,83,US,
146,prainha,-1.8000,-53.4800,80,BR,
38,cartagena del chaira,1.3349,-74.8429,88,CO,
453,ghat,24.9647,10.1728,32,LY,
126,ceeldheer,3.8488,47.1806,88,SO,
425,galliano,29.4422,-90.2992,88,US,
259,mawlaik,23.6333,94.4167,86,MM,


# Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [55]:
# Set parameters to search for a hotel
categories="accommodation.hotel"
conditions= ""
radius = 10000
filters=""
bias=""

params = {
    "categories":categories,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df["Lat"][index]
    longtitude = hotel_df["Lng"][index]
        
    # Add the current city's latitude and longitude to the params dictionary
    filters=f"circle:{longtitude},{latitude},{radius}"
    bias=f"proximity:{longtitude},{latitude}"
    
    params["filter"] = filters
    params["bias"] = bias

    #print(params)

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response=requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    #print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
sample_data = filtered_cities_df.sample(20)  # Adjust the number of samples as needed
display(sample_data)

Starting hotel search
stephenville - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
urucara - nearest hotel: No hotel found
cartagena del chaira - nearest hotel: Los Corceles
hawaiian paradise park - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
ascension - nearest hotel: Cajun Country Inn
san andres - nearest hotel: Hotel Decameron Isleño
jamaame - nearest hotel: No hotel found
dzilam gonzalez - nearest hotel: No hotel found
puerto armuelles - nearest hotel: Coco del Mar
rebola - nearest hotel: Preguis hotel
nova sintra - nearest hotel: Residência Ka Dencho
ceeldheer - nearest hotel: No hotel found
enoch - nearest hotel: Travelodge Cedar City
prainha - nearest hotel: No hotel found
wisconsin rapids - nearest hotel: Hotel Mead
kolda - nearest hotel: le firdou
bonthe - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
zaltan - nearest hotel: No hotel found
padangsidempuan - nearest hotel: Hotel Mega

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
447,447,cidade velha,14.9167,-23.6167,26.26,89,20,3.09,CV,1727752612
543,543,puerto maldonado,-12.6000,-69.1833,25.38,63,63,1.25,PE,1727752716
87,87,san andres,12.5847,-81.7006,26.01,89,20,1.54,CO,1727752201
54,54,wailua homesteads,22.0669,-159.3780,26.53,80,100,3.09,US,1727752164
379,379,wamba,8.9333,8.6000,21.70,97,52,1.01,NG,1727752538
224,224,san jose del guaviare,2.5729,-72.6459,23.66,77,27,0.52,CO,1727752357
488,488,goldsboro,35.3849,-77.9928,22.06,99,75,2.06,US,1727752656
278,278,uturoa,-16.7333,-151.4333,26.78,75,45,3.48,PF,1727752421
102,102,puerto armuelles,8.2833,-82.8667,24.49,88,100,0.45,PA,1727752217
357,357,crestview,30.7621,-86.5705,24.58,90,40,0.00,US,1727752373


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [57]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng", 
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    scale= 1.0,
    color="City",
    hover_cols=["Humidity","Hotel Name","Country"]
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)